Data:

https://www.kaggle.com/code/roydatascience/us-election-results-2020/input?select=president_county_candidate.csv

In [1]:
import numpy as np 
import pandas as pd 
import plotly.figure_factory as ff
import plotly.express as px
import time
from datetime import datetime
import kaleido

data = pd.read_csv('/Users/kellyg/maps-project/data/president_state.csv')
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
datafile = data.merge(state_codes, left_on='state', right_on='usa_state')

fig = px.choropleth(datafile, 
                    locations='usa_state_code', 
                    color="total_votes",
                    color_continuous_scale="PuOr", 
                    range_color=(0, 10000000),
                    locationmode = 'USA-states',  
                    scope="usa",
                    title='USA Presidential Votes Counts' 
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# fig.write_image("votes.png")

fig.show()


![Figure](map_png/votes.png)

In [2]:
import pandas as pd
import plotly.express as px

data = pd.read_csv('/Users/kellyg/maps-project/data/president_state.csv')
state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
pop = pd.read_csv('/Users/kellyg/maps-project/data/states_pop.csv')
datafile = data.merge(state_codes, left_on='state', right_on='usa_state')

votes_percapita = datafile.merge(pop, on='usa_state_code')

#votes per capita
votes_percapita['votes_per_capita'] = votes_percapita['total_votes'] / votes_percapita['population']

fig = px.choropleth(votes_percapita, 
                    locations='usa_state_code', 
                    color="votes_per_capita", 
                    color_continuous_scale="PuOr", 
                    locationmode='USA-states',  
                    scope="usa",
                    title='USA Presidential Votes Per Capita' 
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
# fig.write_image("vote_capita.png")

fig.show()


![Figure](map_png/vote_capita.png)

# Note:

Maps are interactive (zoom and hover for numerical data) when jupyter files ran, but GitHub does not support this feature, so png versions of maps were added. 

In [3]:

import warnings
warnings.filterwarnings("ignore")

df_president_county = pd.read_csv('/Users/kellyg/maps-project/data/president_county_candidate.csv')
df_president_county = df_president_county[df_president_county.party.isin(['DEM','REP'])]
df_president_county = df_president_county.groupby(['state','party'])["total_votes"].sum()
df_president_county = df_president_county.reset_index()

state_codes = pd.read_csv('/Users/kellyg/maps-project/data/world_country_and_usa_states_latitude_and_longitude_values.csv')
state_codes = state_codes[['usa_state','usa_state_code']]
df_president_county = df_president_county.merge(state_codes, left_on='state', right_on='usa_state')
df_president_county = df_president_county.drop(['usa_state'], axis=1)
df_president_county_dummy = pd.get_dummies(df_president_county['party'])

df_president_county = df_president_county.join(df_president_county_dummy)
df_president_county['DEM_votes'] = df_president_county['DEM'] * df_president_county['total_votes'] 
df_president_county['REP_votes'] = df_president_county['REP'] * df_president_county['total_votes'] 
df_president_county = df_president_county.groupby(['state', 'usa_state_code'])[["DEM_votes", "REP_votes"]].sum()
df_president_county = df_president_county.reset_index()
df_president_county['percent_democrat'] = df_president_county['DEM_votes']*100/(df_president_county['REP_votes']+df_president_county['DEM_votes'])

fig = px.choropleth(df_president_county, 
                    locations="usa_state_code", 
                    color = "percent_democrat",
                    locationmode = 'USA-states', 
                    hover_name="state",
                    range_color=[25,75],
                    color_continuous_scale = 'RdBu',
                    scope="usa",
                    title='2020 Votes By Party')

# fig.write_image("vote_party.png")

fig.show()

![Figure](map_png/vote_party.png)

In [56]:
import pandas as pd
import plotly.express as px

# Load electoral vote data
df_electoral_votes = pd.read_csv('/Users/kellyg/maps-project/data/electoral_college.csv')
df_electoral_votes.columns = df_electoral_votes.columns.str.strip()
df_electoral_votes.rename(columns={'State': 'state'}, inplace=True)

# Merge electoral votes with president county data
df_president_county = pd.merge(df_president_county, df_electoral_votes, on='state', how='left', suffixes=('', '_electoral'))
df_president_county['total_votes'] = df_president_county['DEM_votes'] + df_president_county['REP_votes']

# Fill NaN values in 'Electoral Votes' with 3 for DC
df_president_county['Electoral Votes'] = df_president_county.apply(
    lambda row: 3 if pd.isna(row['Electoral Votes']) and row['state'] == 'District of Columbia' else row['Electoral Votes'],
    axis=1
)

# Calculate electoral vote weight
df_president_county['electoral_vote_weight'] = df_president_county['Electoral Votes'] / df_president_county['total_votes']

# Calculate turnout percentage
df_president_county['turnout_percentage'] = df_president_county['total_votes'] / df_president_county['total_votes'].max()

# Turnout influence (turnout % * electoral vote weight)
df_president_county['turnout_influence'] = df_president_county['turnout_percentage'] * df_president_county['electoral_vote_weight']

# Calculate margin of victory
df_president_county['winner'] = df_president_county.apply(lambda row: 'DEM' if row['DEM_votes'] > row['REP_votes'] else 'REP', axis=1)
df_president_county['margin_of_victory'] = df_president_county['DEM_votes'] - df_president_county['REP_votes']
df_president_county['abs_margin_of_victory'] = df_president_county['margin_of_victory'].abs()

# Weighted margin of victory
df_president_county['weighted_margin_of_victory'] = df_president_county['abs_margin_of_victory'] * df_president_county['Electoral Votes']

# Calculate party influence weight
df_president_county['party_influence_weight'] = df_president_county['abs_margin_of_victory'] * df_president_county['Electoral Votes']

# Calculate the Vote Impact Score
df_president_county['vote_impact_score'] = (
    (df_president_county['electoral_vote_weight'] * (1 - df_president_county['turnout_percentage'])) /
    df_president_county['abs_margin_of_victory']
) * df_president_county['party_influence_weight']

# Scale Vote Impact Score
df_president_county['scaled_vote_impact_score'] = df_president_county['vote_impact_score'] * 10000000

# Check the scaled_vote_impact_score values
print(df_president_county[['state', 'scaled_vote_impact_score']].head(50))

# Create a Plotly choropleth map
fig = px.choropleth(
    df_president_county, 
    locations='state',  
    locationmode="USA-states",  
    color='scaled_vote_impact_score', 
    hover_name='state', 
    hover_data=['scaled_vote_impact_score'], 
    color_continuous_scale='Viridis', 
    labels={'scaled_vote_impact_score': 'Vote Impact Score'},
    title="Vote Impact Score by State"
)

# Adjust layout to focus on the United States
fig.update_geos(
    visible=True,
    resolution=110,
    scope="usa",  
    projection_type="albers usa"  
)

# Show the map
fig.show()


                   state  scaled_vote_impact_score
0                Alabama                306.260895
1                 Alaska                256.905343
2                Arizona                292.250807
3               Arkansas                282.872149
4             California                  0.000000
5               Colorado                208.279541
6            Connecticut                244.204269
7               Delaware                175.875211
8   District of Columbia                262.671385
9                Florida                275.570573
10               Georgia                369.122423
11                Hawaii                274.846730
12                 Idaho                180.869909
13              Illinois                331.501721
14               Indiana                336.390490
15                  Iowa                196.261833
16                Kansas                252.363183
17              Kentucky                267.497153
18             Louisiana       